# 文本预处理
读取一本小说

In [1]:
import collections
import re
from d2l import torch as d2l

d2l.DATA_HUB['time_machine'] = (d2l.DATA_URL + 'timemachine.txt',
                                '090b5e7e70c295757f55df93cb0a180b9691891a')

def read_time_machine():  
    """Load the time machine dataset into a list of text lines."""
    with open(d2l.download('time_machine'), 'r') as f:
        lines = f.readlines()
    return [re.sub('[^A-Za-z]+', ' ', line).strip().lower() for line in lines] # 将非大写小写的字母全部变成空格，再将大写变成小写，实际中不会暴力移除

lines = read_time_machine()
print(f'text lines：', len(lines))
print(lines[0])
print(lines[10])

文件已存在，路径为: ../data\timemachine.txt
text lines： 3221
the time machine by h g wells
twinkled and his usually pale face was flushed and animated the


每个文本序列又被拆分成一个标记列表

In [2]:
def tokenize(lines, token='word'):  
    """将文本行拆分为单词或字符标记。"""
    if token == 'word':
        return [line.split() for line in lines] # 按照空格分割
    elif token == 'char':
        return [list(line) for line in lines] # 把每个字符拆开，返回一个每一行是一个字符的二维列表
    else:
        print('错误：未知令牌类型：' + token)

tokens = tokenize(lines)
for i in range(11):
    print(tokens[i])

['the', 'time', 'machine', 'by', 'h', 'g', 'wells']
[]
[]
[]
[]
['i']
[]
[]
['the', 'time', 'traveller', 'for', 'so', 'it', 'will', 'be', 'convenient', 'to', 'speak', 'of', 'him']
['was', 'expounding', 'a', 'recondite', 'matter', 'to', 'us', 'his', 'grey', 'eyes', 'shone', 'and']
['twinkled', 'and', 'his', 'usually', 'pale', 'face', 'was', 'flushed', 'and', 'animated', 'the']


构建一个字典，通常也叫做词表（vocabulary），用来将字符串标记映射到从0开始的数字索引中

In [3]:
class Vocab:  #@save
    """文本词表"""
    def __init__(self, tokens=None, min_freq=0, reserved_tokens=None):
        # reserved_tokens保留特殊token
        if tokens is None:
            tokens = []

        if reserved_tokens is None:
            reserved_tokens = []
        
        # 按出现频率排序
        counter = count_corpus(tokens) # 创建频率表
        self.token_freqs = sorted(counter.items(), key=lambda x: x[1],
                                   reverse=True) # 最频繁的放在最前面
        # 未知词元的索引为0
        self.unk, uniq_tokens = 0, ['<unk>'] + reserved_tokens # 创建一个列表，包含unk和所有保留字符串
        
        uniq_tokens += [ # 将所有的不重复的token加入列表，且只要超过最低出现频率的
            token for token, freq in self.token_freqs
            if freq >= min_freq and token not in uniq_tokens]
        
        # token的标号就是索引；字典查询token获得标号
        self.idx_to_token, self.token_to_idx = [], dict()
        for token in uniq_tokens:
            self.idx_to_token.append(token)
            self.token_to_idx[token] = len(self.idx_to_token) - 1

    def __len__(self):
        return len(self.idx_to_token)

    def __getitem__(self, tokens):
        if not isinstance(tokens, (list, tuple)):
            return self.token_to_idx.get(tokens, self.unk)
        return [self.__getitem__(token) for token in tokens]

    def to_tokens(self, indices):
        if not isinstance(indices, (list, tuple)):
            return self.idx_to_token[indices]
        return [self.idx_to_token[index] for index in indices]

    
    def unk(self):  # 未知词元的索引为0
        return 0

    
    def token_freqs(self):
        return self._token_freqs

def count_corpus(tokens):  #@save
    """统计词元的频率"""
    # 这里的tokens是1D列表或2D列表
    if len(tokens) == 0 or isinstance(tokens[0], list):
        # 将词元列表展平成一个列表
        tokens = [token for line in tokens for token in line]
    return collections.Counter(tokens) # 统计每一个独一无二的token的出现次数
    # 键为token 值为次数

创建词汇表

In [4]:
vocab = Vocab(tokens)
print(list(vocab.token_to_idx.items())[:10])

[('<unk>', 0), ('the', 1), ('i', 2), ('and', 3), ('of', 4), ('a', 5), ('to', 6), ('was', 7), ('in', 8), ('that', 9)]


In [5]:
for i in [0, 10]:
    print('words:', tokens[i])
    print('indices:', vocab[tokens[i]])

words: ['the', 'time', 'machine', 'by', 'h', 'g', 'wells']
indices: [1, 19, 50, 40, 2183, 2184, 400]
words: ['twinkled', 'and', 'his', 'usually', 'pale', 'face', 'was', 'flushed', 'and', 'animated', 'the']
indices: [2186, 3, 25, 1044, 362, 113, 7, 1421, 3, 1045, 1]


In [6]:
def load_corpus_time_machine(max_tokens=-1):  
    """返回时光机器数据集的标记索引列表和词汇表。"""
    lines = read_time_machine()
    tokens = tokenize(lines, 'char')
    vocab = Vocab(tokens)
    # 一长串的整数，每个整数对应一个token
    corpus = [vocab[token] for line in tokens for token in line] # vocab[] token->index
    if max_tokens > 0:
        corpus = corpus[:max_tokens]
    return corpus, vocab

corpus, vocab = load_corpus_time_machine()
len(corpus), len(vocab), corpus # 按照字符分割只有unk加26个字母


文件已存在，路径为: ../data\timemachine.txt


(170580,
 28,
 [3,
  9,
  2,
  1,
  3,
  5,
  13,
  2,
  1,
  13,
  4,
  15,
  9,
  5,
  6,
  2,
  1,
  21,
  19,
  1,
  9,
  1,
  18,
  1,
  17,
  2,
  12,
  12,
  8,
  5,
  3,
  9,
  2,
  1,
  3,
  5,
  13,
  2,
  1,
  3,
  10,
  4,
  22,
  2,
  12,
  12,
  2,
  10,
  1,
  16,
  7,
  10,
  1,
  8,
  7,
  1,
  5,
  3,
  1,
  17,
  5,
  12,
  12,
  1,
  21,
  2,
  1,
  15,
  7,
  6,
  22,
  2,
  6,
  5,
  2,
  6,
  3,
  1,
  3,
  7,
  1,
  8,
  20,
  2,
  4,
  23,
  1,
  7,
  16,
  1,
  9,
  5,
  13,
  17,
  4,
  8,
  1,
  2,
  24,
  20,
  7,
  14,
  6,
  11,
  5,
  6,
  18,
  1,
  4,
  1,
  10,
  2,
  15,
  7,
  6,
  11,
  5,
  3,
  2,
  1,
  13,
  4,
  3,
  3,
  2,
  10,
  1,
  3,
  7,
  1,
  14,
  8,
  1,
  9,
  5,
  8,
  1,
  18,
  10,
  2,
  19,
  1,
  2,
  19,
  2,
  8,
  1,
  8,
  9,
  7,
  6,
  2,
  1,
  4,
  6,
  11,
  3,
  17,
  5,
  6,
  23,
  12,
  2,
  11,
  1,
  4,
  6,
  11,
  1,
  9,
  5,
  8,
  1,
  14,
  8,
  14,
  4,
  12,
  12,
  19,
  1,
  20,
  4,
  12,
  2,
  1,
